In [0]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
print(tf.__version__)

1.15.0


In [0]:
from keras.datasets import mnist

(x_train_full, y_train_full), (x_test, y_test) = mnist.load_data()

x_train_full, x_test = x_train_full.reshape((-1, 1, 28, 28)) / 255.0, x_test.reshape((-1, 1, 28, 28)) / 255.0

y_train_full = keras.utils.to_categorical(y_train_full)
y_test = keras.utils.to_categorical(y_test)

x_train_full.shape, x_test.shape

Using TensorFlow backend.


11493376/11490434 [==============================] - 2s 0us/step


((60000, 1, 28, 28), (10000, 1, 28, 28))

In [0]:
x_train, x_val = x_train_full[:55000], x_train_full[55000:]
y_train, y_val = y_train_full[:55000], y_train_full[55000:]

In [0]:
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(1, 28, 28)))
model.add(keras.layers.Conv2D(150, (5,5), strides=2, padding="same", activation="relu", data_format="channels_first"))
model.add(keras.layers.Conv2D(300, (3,3), padding="same", activation="relu", data_format="channels_first"))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(50, activation="relu"))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(10, activation="softmax"))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.compile(loss="categorical_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])

In [0]:
history = model.fit(x_train, y_train, epochs=30, 
                    validation_data=(x_val, y_val))

Train on 55000 samples, validate on 5000 samples
Epoch 1/30
55000/55000 [==============================] - 36s 651us/sample - loss: 0.1913 - acc: 0.9438 - val_loss: 0.0444 - val_acc: 0.9878
Epoch 2/30
55000/55000 [==============================] - 30s 541us/sample - loss: 0.0710 - acc: 0.9814 - val_loss: 0.0355 - val_acc: 0.9888
Epoch 3/30
55000/55000 [==============================] - 30s 540us/sample - loss: 0.0509 - acc: 0.9862 - val_loss: 0.0371 - val_acc: 0.9900
Epoch 4/30
55000/55000 [==============================] - 30s 541us/sample - loss: 0.0392 - acc: 0.9897 - val_loss: 0.0386 - val_acc: 0.9906
Epoch 5/30
55000/55000 [==============================] - 30s 538us/sample - loss: 0.0340 - acc: 0.9909 - val_loss: 0.0286 - val_acc: 0.9922
Epoch 6/30
55000/55000 [==============================] - 30s 540us/sample - loss: 0.0268 - acc: 0.9928 - val_loss: 0.0315 - val_acc: 0.9932
Epoch 7/30
55000/55000 [==============================] - 30s 543us/sample - loss: 0.0243 - acc: 0.9934 -

In [0]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 155us/sample - loss: 0.0549 - acc: 0.9926


[0.05487303571678099, 0.9926]

#Ex 10

In [0]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
import tensorflow_datasets as tfds

In [0]:
dataset, info = tfds.load("cifar10", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples
print("Train Size: {}, Test Size: {}".format(train_size, test_size))
class_names = info.features["label"].names
print("Class Names: {}".format(class_names))
n_classes = info.features["label"].num_classes
print("n_classes: {}".format(n_classes))

Train Size: 50000, Test Size: 10000
Class Names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
n_classes: 10


In [0]:
train_set = tfds.load("cifar10", split="train[:40000]")
valid_set = tfds.load("cifar10", split="train[40000:]")
test_set = tfds.load("cifar10", split="test")

In [0]:
#def preprocess2(ex):
#  x = ex["image"]
#  y = ex["label"]
#  #x = tf.cast(x, 'float32')
# x = tf.image.resize(x, [224, 224]) #prepare for resnet
#  x = keras.applications.resnet50.preprocess_input(x)
#  y = tf.one_hot(y, 10)
#  return x, y

def preprocess(ex):
  image = ex["image"]
  label = ex["label"]
  resized_image = tf.image.resize(image, [224, 224])
  final_image = keras.applications.xception.preprocess_input(resized_image)
  label_oh = tf.one_hot(label, n_classes)
  return final_image, label_oh

batch_size = 32
train_set = train_set.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).repeat().prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

In [0]:
for item in tfds.as_numpy(train_set.take(1)):
  print(item)

(array([[[[ 0.16078436,  0.30196083,  0.51372564],
         [ 0.16078436,  0.30196083,  0.51596653],
         [ 0.16078436,  0.30196083,  0.5182073 ],
         ...,
         [-0.31764704, -0.24705875, -0.09019601],
         [-0.31764704, -0.24705875, -0.09019601],
         [-0.31764704, -0.24705875, -0.09019601]],

        [[ 0.16190481,  0.30532217,  0.51932776],
         [ 0.16254508,  0.30532217,  0.5212486 ],
         [ 0.16318536,  0.30532217,  0.5231694 ],
         ...,
         [-0.31316525, -0.23921561, -0.08683467],
         [-0.31316525, -0.23921561, -0.08683467],
         [-0.31316525, -0.23921561, -0.08683467]],

        [[ 0.16302538,  0.30868363,  0.5249301 ],
         [ 0.1643058 ,  0.30868363,  0.52653074],
         [ 0.16558635,  0.30868363,  0.52813137],
         ...,
         [-0.3086834 , -0.23137248, -0.08347338],
         [-0.3086834 , -0.23137248, -0.08347338],
         [-0.3086834 , -0.23137248, -0.08347338]],

        ...,

        [[-0.10588229, -0.16862738, -

In [0]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [0]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)
for layer in base_model.layers:
  layer.trainable = False

In [0]:
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_set, epochs=5, validation_data=valid_set, steps_per_epoch=1250, validation_steps=312)

Train on 1250 steps, validate on 312 steps
Epoch 1/5
1250/1250 [==============================] - 145s 116ms/step - loss: 0.7472 - acc: 0.7629 - val_loss: 0.7440 - val_acc: 0.8115
Epoch 2/5
1250/1250 [==============================] - 141s 113ms/step - loss: 0.5596 - acc: 0.8124 - val_loss: 0.7147 - val_acc: 0.8125
Epoch 3/5
1250/1250 [==============================] - 140s 112ms/step - loss: 0.5324 - acc: 0.8225 - val_loss: 0.6897 - val_acc: 0.8167
Epoch 4/5
1250/1250 [==============================] - 144s 115ms/step - loss: 0.5225 - acc: 0.8234 - val_loss: 0.6788 - val_acc: 0.8170
Epoch 5/5
1250/1250 [==============================] - 143s 114ms/step - loss: 0.5173 - acc: 0.8273 - val_loss: 0.6652 - val_acc: 0.8197


In [0]:
for layer in base_model.layers:
  layer.trainable = True

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_set, epochs=3, validation_data=valid_set, steps_per_epoch=1250, validation_steps=312)

Train on 1250 steps, validate on 312 steps
Epoch 1/3
1250/1250 [==============================] - 543s 434ms/step - loss: 0.2366 - acc: 0.9218 - val_loss: 0.1288 - val_acc: 0.9582
Epoch 2/3
1250/1250 [==============================] - 538s 431ms/step - loss: 0.0483 - acc: 0.9836 - val_loss: 0.1105 - val_acc: 0.9651
Epoch 3/3
1250/1250 [==============================] - 540s 432ms/step - loss: 0.0145 - acc: 0.9954 - val_loss: 0.1198 - val_acc: 0.9675


In [0]:
model.evaluate(test_set)

    313/Unknown - 29s 92ms/step - loss: 0.1370 - acc: 0.9641

[0.13701555625766704, 0.9641]

Pretty decent results for not really tweaking this... :)